#Stage 2   
VIP CUP   
______
**SPA Group**  
Sajjad Jabbarzade   
Parham Soltani    
Amirtaha Aghasi   

______   

##Libraries

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab.patches import cv2_imshow

##Preparing Data   
Importing the video_2.mp4 file into the Google VM Drive

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving video_2.mp4 to video_2.mp4


In [ ]:
!unzip SSD.zip

Archive:  SSD.zip
  inflating: coco_class_labels.txt   
  inflating: tf_text_graph_ssd.py    
   creating: models/
   creating: models/ssd_mobilenet_v2_coco_2018_03_29/
  inflating: models/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb  
  inflating: models/ssd_mobilenet_v2_coco_2018_03_29.pbtxt  


##Functions

In [ ]:
def get_fps(inputvideo):
  return int(inputvideo.get(cv2.CAP_PROP_FPS))

##Task 2

In [ ]:
with open("/content/coco_class_labels.txt") as fp:
  labels = fp.read().split("\n")

In [ ]:
labels[1]

'person'

In [ ]:
labels[3]

'car'

In [ ]:
labels[10]

'traffic light'

In [ ]:
our_labels = [labels[1], labels[3], labels[10]]

In [ ]:
our_labels

['person', 'car', 'traffic light']

In [ ]:
modelFile  = "/content/models/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"
configFile = "/content/models/ssd_mobilenet_v2_coco_2018_03_29.pbtxt"

In [ ]:
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [ ]:
video_input_file_name = "/content/video_2.mp4"

In [ ]:
cap = cv2.VideoCapture(video_input_file_name)

In [ ]:
fps = get_fps(cap)

In [ ]:
video_output_file_name = "Our Output.mp4"
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_out = cv2.VideoWriter(video_output_file_name, cv2.VideoWriter_fourcc(*"XVID"), fps, (width, height))

In [ ]:
while True:
  ret, frame = cap.read()
  if not ret:
    break

  blob = cv2.dnn.blobFromImage(frame, size=(300, 300), swapRB=True)
  net.setInput(blob)
  detections = net.forward()
  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    class_id = int(detections[0, 0, i, 1])
    if confidence > 0.5 and labels[class_id] in our_labels:
      box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
      (startX, startY, endX, endY) = box.astype("int")

      cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

      label = f"{labels[class_id]}: {confidence:.2f}"
      cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

  # cv2_imshow(frame)
  video_out.write(frame)

cap.release()
video_out.release()